In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 5, Finished, Available, Finished)

In [3]:
%run lake_path

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 7, Finished, Available, Finished)

In [4]:
%run bronze_note_RU

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cdaf9fa8-e3e8-4308-99f4-3e78eedda45a)

root
 |-- id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- birthdate: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- race: string (nullable = true)
 |-- department: string (nullable = true)
 |-- jobtitle: string (nullable = true)
 |-- location: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- termdate: timestamp (nullable = true)
 |-- location_city: string (nullable = true)
 |-- location_state: string (nullable = true)

Number of rows  22227


SynapseWidget(Synapse.DataFrame, b08a0a5a-bca0-4033-978f-ad474c990a73)

SynapseWidget(Synapse.DataFrame, f93c9871-75a8-4c16-87cf-d83e1f8c2c26)

HR data successfully moved to the Bronze Layer: 'hr_data_bronze.parquet'


In [5]:
# Bronze Layer Data
bronze_df = spark.read.parquet(bronze_file_path)

# Show sample data
bronze_df.show(5)

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 15, Finished, Available, Finished)

+----------+----------+---------+----------+------+--------------------+---------------+--------------------+------------+---------+--------------------+-------------+--------------+
|        id|first_name|last_name| birthdate|gender|                race|     department|            jobtitle|    location|hire_date|            termdate|location_city|location_state|
+----------+----------+---------+----------+------+--------------------+---------------+--------------------+------------+---------+--------------------+-------------+--------------+
|00-0381660|Wainwright| Corfield|12/13/1996|     M|               Asian|    Engineering|Business Systems ...|Headquarters|2/20/2001|2008-12-05T01:21:...|    Cleveland|          Ohio|
|00-1100714|     Regen|  Nafzger|  4/6/1990|     M|               White|Human Resources|    Senior Recruiter|Headquarters|7/23/2016|2022-07-12T02:26:...|    Cleveland|          Ohio|
|00-1268049|       Fay| Monnelly|  7/9/1966|     M|Native Hawaiian o...|    Engineeri

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta.tables import DeltaTable

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 16, Finished, Available, Finished)

In [7]:
# Initialize Spark Session with Delta support
spark = SparkSession.builder \
    .appName("HR Silver Layer") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .enableHiveSupport() \
    .getOrCreate()

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 17, Finished, Available, Finished)

In [8]:
# Step 2: Count Total Records and Null Values Before Processing
total_records_before = bronze_df
null_counts_before = {col_name: bronze_df.filter(col(col_name).isNull()).count() for col_name in bronze_df.columns}

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 18, Finished, Available, Finished)

In [9]:
# Step 3: Remove Nulls (except termdate)
columns_to_clean = [col_name for col_name in bronze_df.columns if col_name != "termdate"]
df_cleaned = bronze_df.dropna(subset=columns_to_clean)
df_cleaned.show()


StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 19, Finished, Available, Finished)

+----------+----------+----------+----------+------+--------------------+--------------------+--------------------+------------+----------+--------------------+-------------+--------------+
|        id|first_name| last_name| birthdate|gender|                race|          department|            jobtitle|    location| hire_date|            termdate|location_city|location_state|
+----------+----------+----------+----------+------+--------------------+--------------------+--------------------+------------+----------+--------------------+-------------+--------------+
|00-0381660|Wainwright|  Corfield|12/13/1996|     M|               Asian|         Engineering|Business Systems ...|Headquarters| 2/20/2001|2008-12-05T01:21:...|    Cleveland|          Ohio|
|00-1100714|     Regen|   Nafzger|  4/6/1990|     M|               White|     Human Resources|    Senior Recruiter|Headquarters| 7/23/2016|2022-07-12T02:26:...|    Cleveland|          Ohio|
|00-1268049|       Fay|  Monnelly|  7/9/1966|     

In [10]:
# Step 4: Remove Duplicates
df_silver = df_cleaned.dropDuplicates()
df_silver.show()

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 20, Finished, Available, Finished)

+----------+----------+-----------+----------+------+--------------------+------------------+--------------------+------------+----------+--------------------+-------------+--------------+
|        id|first_name|  last_name| birthdate|gender|                race|        department|            jobtitle|    location| hire_date|            termdate|location_city|location_state|
+----------+----------+-----------+----------+------+--------------------+------------------+--------------------+------------+----------+--------------------+-------------+--------------+
|27-0048592|      Holt|Bottelstone| 9/18/2001|     M|Native Hawaiian o...|        Accounting|     Cost Accountant|Headquarters| 9/22/2008|2014-06-28T13:32:...|    Cleveland|          Ohio|
|67-8695740|     Gilly|       Taig|  7/9/1971|     M|               White|             Legal|Administrative As...|Headquarters| 5/16/2002|2008-02-23T03:41:...|    Cleveland|          Ohio|
|40-8578044|     Elisa|     Bishop|10/22/1977|    FM|  

In [11]:
# Step 5:  Null Value count
null_counts_before = {c: bronze_df.filter(col(c).isNull()).count() for c in bronze_df.columns}


# Step 6: Count Total Records After Processing
total_records_before = bronze_df.count()

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 21, Finished, Available, Finished)

In [12]:
# Step 7: Count after processing
total_records_after = df_silver.count()
nulls_removed = total_records_before - df_cleaned.count()  #  Get count before subtracting
duplicates_removed = df_cleaned.count() - total_records_after  #  Get count before subtracting

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 22, Finished, Available, Finished)

In [13]:
# Step 8: Show summary log
print(f"""
Silver Layer Processing Summary:

Total records before processing: {total_records_before}
Null values removed (except termdate): {nulls_removed}
Duplicate records removed: {duplicates_removed}
Total records after cleaning: {total_records_after}
""")

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 23, Finished, Available, Finished)


Silver Layer Processing Summary:

Total records before processing: 22227
Null values removed (except termdate): 0
Duplicate records removed: 13
Total records after cleaning: 22214



In [14]:
# Create a new column 'id_cleaned' by removing dashes from 'id'
from pyspark.sql.functions import *
cleand_id = df_silver.withColumn("int_id", regexp_replace(col("id"), "-", ""))
silver_df = cleand_id.withColumn("int_id", col("int_id").cast("int"))
# silver_df.select("id", "id_cleaned").show(10)
display(silver_df)


StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5688cd42-3538-4518-b497-d524cf3e15b5)

In [15]:
# Convert 'birthdate' and 'hire_date' into proper date format
silver_df_birth_date = silver_df.withColumn("birthdate", to_date(col("birthdate"), "M/d/yyyy"))
hr_data_silver = silver_df_birth_date.withColumn("hire_date",to_date(col("hire_date"),"M/d/yyyy"))
# display(silver_df_hire_date)
# display(silver_df_birth_date)
hr_data_silver.printSchema()

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 25, Finished, Available, Finished)

root
 |-- id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- birthdate: date (nullable = true)
 |-- gender: string (nullable = true)
 |-- race: string (nullable = true)
 |-- department: string (nullable = true)
 |-- jobtitle: string (nullable = true)
 |-- location: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- termdate: string (nullable = true)
 |-- location_city: string (nullable = true)
 |-- location_state: string (nullable = true)
 |-- int_id: integer (nullable = true)



In [16]:
# Change int_id to employee_id

hr_data_silver = hr_data_silver.withColumnRenamed("int_id","employee_id") 
# display(hr_data_silver)

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 26, Finished, Available, Finished)

In [17]:
hr_data_silver.write.format("delta").mode("overwrite").save("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/silver_lake_RU.Lakehouse/Tables")

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 27, Finished, Available, Finished)

In [18]:
df = spark.read.format("delta").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/silver_lake_RU.Lakehouse/Tables")
display(df)

StatementMeta(, a0862b6a-1bff-4428-837a-6ddbf2b47151, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2a3f6b31-b231-4b3c-b969-e0cfd035abbf)